In [1]:
import torch
import torchvision
import numpy as np
from torch import nn,optim
import pandas as pd

In [2]:
train_data=pd.read_csv(r'F:\study\ml\DataSet\House_Prices\train.csv')
test_data=pd.read_csv(r'F:\study\ml\DataSet\House_Prices\test.csv')

In [3]:
train_data[:5]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
print(train_data.shape)
print(test_data.shape)

(1460, 81)
(1459, 80)


In [5]:
all_features=pd.concat((train_data.iloc[:,1:-1],test_data.iloc[:,1:]))
all_features.shape

(2919, 79)

In [7]:
numeric_features=all_features.dtypes[all_features.dtypes !='object'].index
numeric_features

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold'],
      dtype='object')

In [8]:
all_features[numeric_features]=all_features[numeric_features].apply(
lambda x:(x-x.mean()) /x.std())

In [9]:
all_features.isnull().sum()

MSSubClass         0
MSZoning           4
LotFrontage      486
LotArea            0
Street             0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           1
SaleCondition      0
Length: 79, dtype: int64

In [10]:
all_features=all_features.fillna(0)
all_features.shape

(2919, 79)

In [11]:
all_features1=all_features[numeric_features]
all_features1.shape

(2919, 36)

In [12]:
n_train=train_data.shape[0]
train_features=torch.Tensor(all_features1.iloc[:n_train,:].values)
test_features=torch.Tensor(all_features1.iloc[n_train:,:].values)
train_labels=torch.Tensor(train_data.SalePrice.values).view(-1,1)

In [13]:
train_features.shape

torch.Size([1460, 36])

In [14]:
loss=nn.MSELoss()

In [15]:
def get_net(feature_num):
    net=nn.Linear(feature_num,1)
    for p in net.parameters():
        nn.init.normal_(p,mean=0,std=0.01)
    return net

In [16]:
def log_rmse(net,features,labels):
    with torch.no_grad():
        clipped_pred=torch.max(net(features),torch.Tensor([1.]))
        rmse=torch.sqrt(2*(loss(clipped_pred.log(),labels.log())).mean())
    return rmse.item()

In [17]:
def train(net,train_features,train_labels,test_features,
         test_labels,num_epochs,lr,weight_decay,batch_size):
    train_l,test_l=[],[]
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    train_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)
    optimizer=optim.Adam(net.parameters(),lr,weight_decay=weight_decay)
    
    for i in range(num_epochs):
        for x,y in train_iter:
            l=loss(net(x),y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
        train_l.append(log_rmse(net,train_features,train_labels))
        if test_labels is not None:
            test_l.append(log_rmse(net,test_features,test_labels))
    return train_l,test_l

In [18]:
def get_k_fold_data(k,i,x,y):
    assert k>1
    fold_size=x.shape[0]//k
    x_train,y_train=None,None
    data_idx=list(range(x.shape[0]))
    np.random.shuffle(data_idx)
    for j in range(k):
        idx=slice(j*fold_size,(j+1) * fold_size)
        x_part,y_part=x[data_idx[idx],:],y[data_idx[idx]]
        
        if j==i:
            x_valid,y_valid=x_part,y_part
        elif x_train is None:
            x_train,y_train=x_part,y_part
        else:
            x_train=torch.cat((x_train,x_part))
            y_train=torch.cat((y_train,y_part))
    return x_train,y_train,x_valid,y_valid

In [24]:
def k_fold(k,x_train,y_train,num_epochs,lr,weight_decay,batch_size):
    train_l_sum,valid_l_sum=0,0
    for i in range(k):
        data=get_k_fold_data(k,i,x_train,y_train)
        net=get_net(x_train.shape[1])
        train_l,valid_l=train(net,*data,num_epochs,lr,weight_decay,batch_size)
        train_l_sum+=train_l[-1]
        valid_l_sum+=valid_l[-1]
        torch.save(net.state_dict(),r'F:\study\ml\rknn\models\hp_test2.pth')
        print('fold %d,train rmse %f,valid %f'  % (
        i+1,train_l[-1],valid_l[-1]))
    return train_l_sum,valid_l_sum

In [25]:
k,num_epochs,lr,weight_decay,batch_size=5,100,5,0,64

In [26]:
train_l_sum,valid_l_sum=k_fold(k,train_features,train_labels,num_epochs,
                          lr,weight_decay,batch_size)
print('%d fold validation : avg train rmse %f, avg valid rmse %f' % (
k,train_l_sum/k,valid_l_sum/k))

fold 1,train rmse 11.418023,valid 11.096972
fold 2,train rmse 11.265212,valid 11.759585
fold 3,train rmse 11.509514,valid 11.203047
fold 4,train rmse 11.210003,valid 11.995752
fold 5,train rmse 11.420617,valid 11.524381
5 fold validation : avg train rmse 11.364674, avg valid rmse 11.515948


In [27]:
net1=get_net(train_features.shape[1])
net1.load_state_dict(torch.load(r'F:\study\ml\rknn\models\hp_test2.pth'))

<All keys matched successfully>

In [34]:
test_features.shape

torch.Size([1459, 36])

In [29]:
test_features[0]

tensor([-0.8735,  0.4581,  0.1843, -0.7724,  0.3912, -0.3405, -1.1134, -0.5699,
         0.0583,  0.5580, -0.6615, -0.3852, -0.6718, -0.7849, -0.1012, -1.1951,
        -0.8193, -0.2498, -1.0272, -0.7562, -1.0456, -0.2077, -0.9249, -0.9242,
        -0.6692, -1.0066,  1.1937,  0.3659, -0.7027, -0.3595, -0.1033,  1.8499,
        -0.0631, -0.0896, -0.0785,  1.6786])

In [37]:
test_features[3]

tensor([ 0.0673,  0.3724, -0.0241, -0.0632,  0.3912,  0.8810,  0.6574, -0.4584,
         0.3524, -0.2930, -0.5387, -0.2854, -0.5953,  0.7966, -0.1012,  0.2040,
        -0.8193, -0.2498,  0.7812,  1.2324,  0.1699, -0.2077,  0.3495,  0.6235,
         0.7776,  0.3064, -0.0133,  2.1046, -0.1700, -0.3595, -0.1033, -0.2859,
        -0.0631, -0.0896, -0.0785,  1.6786])

In [36]:
net1.eval()
net1(test_features[3]).item()

41125.296875

In [33]:
net1.eval()
trace_model=torch.jit.trace(net1,train_features[0])
trace_model.save(r'F:\study\ml\rknn\models\hp_test2.pt')